# 🎥 Colab Video Compressor

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AdittyaMondal/ColabVideoCompressor/blob/main/colab_notebook.ipynb)

Advanced video compression bot with GPU acceleration and multiple presets.

In [ ]:
#@title 🚀 Enhanced GPU-Accelerated Video Compression Bot
#@markdown ### 🔧 API Configuration
APP_ID = ""  #@param {type:"string"}
API_HASH = ""  #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}
OWNER = "" #@param {type:"string"}
THUMBNAIL = "https://graph.org/file/75ee20ec8d8c8bba84f02.jpg" #@param {type:"string"}

#@markdown ---
#@markdown ### 📊 Compression Presets
preset_choice = "⚡ Ultra Fast (480p)" #@param ["⚡ Ultra Fast (480p)", "🏃 Fast (720p)", "⚖️ Balanced (1080p)", "💎 High Quality (1080p)", "🎬 Cinema Quality (1440p)", "🖥️ Desktop (1080p 60fps)", "📱 Mobile Optimized (720p)", "🎮 Gaming (1080p 60fps)", "🎭 Animation (1080p)", "🎥 Film Grade (4K)"]

#@markdown ---
#@markdown ### 🎨 Output Settings
FILENAME_TEMPLATE = "{original_name} [{preset}]" #@param {type:"string"}
#@markdown Available placeholders: `{original_name}`, `{preset}`, `{resolution}`, `{codec}`, `{date}`, `{time}`

watermark_enabled = False #@param {type:"boolean"}
WATERMARK_TEXT = "Compressed by Bot" #@param {type:"string"}
watermark_position = "bottom-right" #@param ["top-left", "top-right", "bottom-left", "bottom-right", "center"]

#@markdown ---
#@markdown ### ⚙️ Advanced Configuration
MAX_FILE_SIZE = 4000 #@param {type:"integer"}
MAX_QUEUE_SIZE = 15 #@param {type:"integer"}
PROGRESS_UPDATE_INTERVAL = 5 #@param {type:"integer"}
AUTO_DELETE_ORIGINAL = False #@param {type:"boolean"}
ENABLE_HARDWARE_ACCELERATION = True #@param {type:"boolean"}

import os
import subprocess
import json
from datetime import datetime
from IPython.display import HTML, display, clear_output

# Enhanced NVENC/x264 presets with multiple quality options
PRESET_PARAMS = {
    "⚡ Ultra Fast (480p)": {
        "preset": "p1", "qp": 35, "scale": 480, "ab": "96k", "fps": 30,
        "codec": "h264_nvenc", "profile": "baseline", "level": "3.1",
        "description": "Fastest compression, smallest file size"
    },
    "🏃 Fast (720p)": {
        "preset": "p2", "qp": 28, "scale": 720, "ab": "128k", "fps": 30,
        "codec": "h264_nvenc", "profile": "main", "level": "3.1",
        "description": "Good balance of speed and quality"
    },
    "⚖️ Balanced (1080p)": {
        "preset": "p3", "qp": 26, "scale": 1080, "ab": "192k", "fps": 30,
        "codec": "h264_nvenc", "profile": "high", "level": "4.0",
        "description": "Best overall quality/size ratio"
    },
    "💎 High Quality (1080p)": {
        "preset": "p4", "qp": 22, "scale": 1080, "ab": "256k", "fps": 30,
        "codec": "h264_nvenc", "profile": "high", "level": "4.0",
        "description": "High quality with moderate file size"
    },
    "🎬 Cinema Quality (1440p)": {
        "preset": "p5", "qp": 20, "scale": 1440, "ab": "320k", "fps": 24,
        "codec": "h265_nvenc", "profile": "main", "level": "5.0",
        "description": "Cinema-grade quality with HEVC"
    },
    "🖥️ Desktop (1080p 60fps)": {
        "preset": "p3", "qp": 24, "scale": 1080, "ab": "192k", "fps": 60,
        "codec": "h264_nvenc", "profile": "high", "level": "4.2",
        "description": "Optimized for desktop/screen recordings"
    },
    "📱 Mobile Optimized (720p)": {
        "preset": "p2", "qp": 30, "scale": 720, "ab": "128k", "fps": 30,
        "codec": "h264_nvenc", "profile": "baseline", "level": "3.1",
        "description": "Optimized for mobile devices"
    },
    "🎮 Gaming (1080p 60fps)": {
        "preset": "p4", "qp": 23, "scale": 1080, "ab": "256k", "fps": 60,
        "codec": "h264_nvenc", "profile": "high", "level": "4.2",
        "description": "High quality for gaming content"
    },
    "🎭 Animation (1080p)": {
        "preset": "p5", "qp": 18, "scale": 1080, "ab": "192k", "fps": 24,
        "codec": "h265_nvenc", "profile": "main", "level": "4.0",
        "description": "Optimized for animated content"
    },
    "🎥 Film Grade (4K)": {
        "preset": "p6", "qp": 16, "scale": 2160, "ab": "384k", "fps": 24,
        "codec": "h265_nvenc", "profile": "main10", "level": "5.1",
        "description": "Professional grade 4K quality"
    }
}

def run_shell(cmd, show_output=True):
    """Enhanced shell command runner with better error handling."""
    try:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True, timeout=300)
        if process.returncode != 0:
            if show_output:
                print(f"❌ Command failed: {cmd}")
                print(f"Error: {process.stderr}")
            return False, process.stderr
        else:
            if show_output and process.stdout.strip():
                print(process.stdout.strip())
            return True, process.stdout
    except subprocess.TimeoutExpired:
        print(f"⏰ Command timed out: {cmd}")
        return False, "Command timed out"
    except Exception as e:
        print(f"❌ Exception running command: {str(e)}")
        return False, str(e)

def check_gpu_support():
    """Check for NVIDIA GPU support."""
    print("🔍 Checking GPU support...")
    success, output = run_shell("nvidia-smi --query-gpu=name --format=csv,noheader,nounits", show_output=False)
    
    if success and output.strip():
        gpu_name = output.strip().split('\n')[0]
        print(f"✅ NVIDIA GPU detected: {gpu_name}")
        return True
    else:
        print("⚠️  No NVIDIA GPU detected. Will use CPU encoding (slower).")
        return False

def create_enhanced_config():
    """Create enhanced configuration with all settings."""
    params = PRESET_PARAMS[preset_choice]
    
    # Generate filename with placeholders
    now = datetime.now()
    filename_mapping = {
        'preset': preset_choice.split(' ')[1] if ' ' in preset_choice else preset_choice,
        'resolution': f"{params['scale']}p",
        'codec': params['codec'].replace('_nvenc', ''),
        'date': now.strftime('%Y-%m-%d'),
        'time': now.strftime('%H-%M-%S')
    }
    
    # Watermark settings
    watermark_config = {
        'enabled': watermark_enabled,
        'text': WATERMARK_TEXT,
        'position': watermark_position
    } if watermark_enabled else {'enabled': False}
    
    config = {
        # API Settings
        'APP_ID': APP_ID,
        'API_HASH': API_HASH,
        'BOT_TOKEN': BOT_TOKEN,
        'OWNER': OWNER,
        'THUMBNAIL': THUMBNAIL,
        
        # File Settings
        'MAX_FILE_SIZE': MAX_FILE_SIZE,
        'MAX_QUEUE_SIZE': MAX_QUEUE_SIZE,
        'FILENAME_TEMPLATE': FILENAME_TEMPLATE,
        'AUTO_DELETE_ORIGINAL': AUTO_DELETE_ORIGINAL,
        
        # Video Settings
        'V_PRESET': params['preset'],
        'V_QP': params['qp'],
        'V_SCALE': params['scale'],
        'V_FPS': params['fps'],
        'V_CODEC': params['codec'],
        'V_PROFILE': params['profile'],
        'V_LEVEL': params['level'],
        'A_BITRATE': params['ab'],
        
        # Advanced Settings
        'PROGRESS_UPDATE_INTERVAL': PROGRESS_UPDATE_INTERVAL,
        'ENABLE_HARDWARE_ACCELERATION': ENABLE_HARDWARE_ACCELERATION,
        'WATERMARK': watermark_config,
        'FILENAME_MAPPING': filename_mapping,
        'PRESET_DESCRIPTION': params['description']
    }
    
    return config

def setup_environment():
    """Set up the environment with enhanced features."""
    print("🏗️  Setting up environment...")
    
    # Check if already in the right directory
    if not os.path.exists("/content/ColabVideoCompressor"):
        print("📥 Cloning repository...")
        success, _ = run_shell("git clone -q https://github.com/AdittyaMondal/ColabVideoCompressor.git")
        if not success:
            raise Exception("Failed to clone repository")
    
    os.chdir("/content/ColabVideoCompressor")
    
    # Install system dependencies
    print("📦 Installing system packages...")
    packages = [
        "ffmpeg", "mediainfo", "nvidia-utils-470",
        "python3-pip", "python3-dev", "build-essential"
    ]
    
    install_cmd = f"apt-get -qq update && apt-get -qq install -y {' '.join(packages)}"
    success, _ = run_shell(install_cmd)
    
    if not success:
        print("⚠️  Some packages may not have installed correctly, but continuing...")
    
    # Create directory structure
    print("📁 Creating directory structure...")
    directories = ["encode", "downloads", "thumb", "temp", "logs"]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        run_shell(f"chmod -R 777 {directory}", show_output=False)
    
    return True

def create_config_files(config):
    """Create configuration files."""
    print("⚙️  Creating configuration files...")
    
    # Create .env file
    env_content = []
    for key, value in config.items():
        if key == 'WATERMARK':
            env_content.append(f"WATERMARK_ENABLED={value['enabled']}")
            if value['enabled']:
                env_content.append(f"WATERMARK_TEXT='{value['text']}'")
                env_content.append(f"WATERMARK_POSITION={value['position']}")
        elif key == 'FILENAME_MAPPING':
            for k, v in value.items():
                env_content.append(f"FILENAME_{k.upper()}={v}")
        elif isinstance(value, bool):
            env_content.append(f"{key}={str(value).lower()}")
        elif isinstance(value, str):
            env_content.append(f"{key}='{value}'")
        else:
            env_content.append(f"{key}={value}")
    
    with open(".env", "w") as f:
        f.write("\n".join(env_content))
    
    # Create config.json for advanced settings
    with open("config.json", "w") as f:
        json.dump(config, f, indent=2)
    
    print("✅ Configuration files created successfully")

def install_dependencies():
    """Install Python dependencies."""
    print("🐍 Installing Python dependencies...")
    
    # Install base requirements
    success, _ = run_shell("pip install -q -r requirements.txt")
    if not success:
        print("⚠️  Some Python packages may not have installed correctly")
    
    # Install additional packages for enhanced features
    additional_packages = [
        "Pillow>=9.0.0",
        "psutil>=5.8.0",
        "asyncio-throttle>=1.0.0",
        "aiofiles>=0.8.0"
    ]
    
    for package in additional_packages:
        run_shell(f"pip install -q {package}", show_output=False)
    
    print("✅ Dependencies installed successfully")

def display_configuration_summary(config):
    """Display a summary of the current configuration."""
    preset_info = PRESET_PARAMS[preset_choice]
    
    summary_html = f"""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                padding: 20px; border-radius: 12px; color: white; margin: 20px 0;
                box-shadow: 0 4px 15px rgba(0,0,0,0.2);'>
        <h2 style='margin: 0 0 15px 0; font-size: 24px;'>🚀 Bot Configuration Summary</h2>
        
        <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 15px;'>
            <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;'>
                <h3 style='margin: 0 0 10px 0; color: #ffd700;'>📊 Compression Settings</h3>
                <p><strong>Preset:</strong> {preset_choice}</p>
                <p><strong>Resolution:</strong> {preset_info['scale']}p @ {preset_info['fps']}fps</p>
                <p><strong>Codec:</strong> {preset_info['codec']}</p>
                <p><strong>Audio:</strong> {preset_info['ab']}</p>
                <p><em>{preset_info['description']}</em></p>
            </div>
            
            <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;'>
                <h3 style='margin: 0 0 10px 0; color: #ffd700;'>⚙️ Bot Settings</h3>
                <p><strong>Max File Size:</strong> {MAX_FILE_SIZE} MB</p>
                <p><strong>Queue Size:</strong> {MAX_QUEUE_SIZE}</p>
                <p><strong>GPU Acceleration:</strong> {'✅ Enabled' if ENABLE_HARDWARE_ACCELERATION else '❌ Disabled'}</p>
                <p><strong>Watermark:</strong> {'✅ Enabled' if watermark_enabled else '❌ Disabled'}</p>
            </div>
        </div>
    </div>
    """
    
    display(HTML(summary_html))

def main_setup():
    """Main setup function with enhanced error handling."""
    try:
        # Validate inputs
        if not all([APP_ID, API_HASH, BOT_TOKEN, OWNER]):
            raise ValueError("❌ All API credentials are required!")
        
        # Clear output for better UX
        clear_output(wait=True)
        
        print("🚀 Enhanced Video Compression Bot Setup")
        print("=" * 50)
        
        # Check GPU support
        gpu_available = check_gpu_support()
        
        # Setup environment
        setup_environment()
        
        # Create configuration
        config = create_enhanced_config()
        
        # Display configuration summary
        display_configuration_summary(config)
        
        # Create config files
        create_config_files(config)
        
        # Install dependencies
        install_dependencies()
        
        print("\n🎉 Setup completed successfully!")
        print("🤖 Starting bot... Check your Telegram bot!")
        print("=" * 50)
        
        # Start the bot
        success, output = run_shell("python3 -m bot")
        
        if not success:
            print(f"❌ Bot failed to start: {output}")
            return False
        
        return True
        
    except Exception as e:
        print(f"❌ Setup failed: {str(e)}")
        return False

#@markdown ---
#@markdown ### 🚀 Run Setup
run_setup = True #@param {type:"boolean"}

if run_setup:
    success = main_setup()
    if not success:
        print("\n🔧 Troubleshooting Tips:")
        print("• Check your API credentials")
        print("• Ensure your bot token is valid")
        print("• Verify you have sufficient system resources")

# Enhanced Information Panel
display(HTML('''
<div style='background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
            padding: 25px; border-radius: 15px; margin: 20px 0;
            box-shadow: 0 8px 25px rgba(0,0,0,0.15);'>
    <h2 style='color: white; margin: 0 0 20px 0; font-size: 28px;'>📚 Enhanced Bot Features</h2>
    
    <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(280px, 1fr)); gap: 20px;'>
        <div style='background: rgba(255,255,255,0.15); padding: 18px; border-radius: 10px; backdrop-filter: blur(10px);'>
            <h3 style='color: #ffd700; margin: 0 0 12px 0;'>🎯 Filename Templates</h3>
            <p style='color: white; margin: 5px 0; font-size: 14px;'><code>{original_name}</code> - Original filename</p>
            <p style='color: white; margin: 5px 0; font-size: 14px;'><code>{preset}</code> - Compression preset</p>
            <p style='color: white; margin: 5px 0; font-size: 14px;'><code>{resolution}</code> - Output resolution</p>
            <p style='color: white; margin: 5px 0; font-size: 14px;'><code>{codec}</code> - Video codec used</p>
            <p style='color: white; margin: 5px 0; font-size: 14px;'><code>{date}</code> - Processing date</p>
            <p style='color: white; margin: 5px 0; font-size: 14px;'><code>{time}</code> - Processing time</p>
        </div>
        
        <div style='background: rgba(255,255,255,0.15); padding: 18px; border-radius: 10px; backdrop-filter: blur(10px);'>
            <h3 style='color: #ffd700; margin: 0 0 12px 0;'>🚀 Performance Features</h3>
            <ul style='color: white; margin: 8px 0; padding-left: 20px; font-size: 14px;'>
                <li>Hardware-accelerated encoding (NVENC)</li>
                <li>Multiple quality presets</li>
                <li>Real-time progress updates</li>
                <li>Queue management system</li>
                <li>Automatic cleanup options</li>
                <li>Watermark support</li>
            </ul>
        </div>
        
        <div style='background: rgba(255,255,255,0.15); padding: 18px; border-radius: 10px; backdrop-filter: blur(10px);'>
            <h3 style='color: #ffd700; margin: 0 0 12px 0;'>🔗 Required Links</h3>
            <p style='color: white; margin: 8px 0; font-size: 14px;'>
                <strong>API Credentials:</strong><br>
                <a href='https://my.telegram.org' target='_blank' style='color: #ffd700;'>my.telegram.org</a>
            </p>
            <p style='color: white; margin: 8px 0; font-size: 14px;'>
                <strong>Bot Token:</strong><br>
                <a href='https://t.me/BotFather' target='_blank' style='color: #ffd700;'>@BotFather</a>
            </p>
            <p style='color: white; margin: 8px 0; font-size: 14px;'>
                <strong>Owner ID:</strong><br>
                <a href='https://t.me/userinfobot' target='_blank' style='color: #ffd700;'>@userinfobot</a>
            </p>
        </div>
    </div>
</div>
'''))